In [ ]:
# -- under construction --
import json

json_data = open('/home/marcel/projects/eco/src/python/notebooks/log-final.json').read()
data = json.loads(json_data)

In [ ]:
# 1

import gensim
from nltk.corpus import stopwords
import numpy as np

model = gensim.models.Word2Vec.load('/mnt/drive1/data/eco/word2vec_models/wiki_plus_v3_valid_combined.txt_numpy.w2vmodel')


# sentence to vector via w2v
def to_vector(sentence):
    sentence = sentence.lower()
    # not using english stop words, only using content words
    words = [word for word in sentence.split() if (word not in stopwords.words("english") and word in model.vocab)]

    vectors = []
    for word in words:
        vectors.append(model[word])

    vector = np.mean(vectors, axis=0)
    return vector


# converts a v4 text file into feather file with w2v vec saved
def v4_txt_to_feather(path, feather_name):
    data = open(path).readlines()

    row_list = []
    for line in data:
        sentence = line.split(';')[0].rstrip()
        vec = to_vector(sentence)
        new_row = {}
        index = 0
        try:
            for i in vec:
                new_row['p'+str(index)] = i
                index += 1

            new_row['sentence'] = sentence
            row_list.append(new_row)
        except:
            # only happening sometimes. \o/
            pass

    df = pd.DataFrame(row_list)
    df.to_feather(feather_name)

    
v4_txt_to_feather('/mnt/drive1/data/eco/NAIL_DATAFIELD_txt/parsed_v4/own_collection_valid.txt', 'out.feather')

In [ ]:
# 2

import pandas as pd
import feather

df = feather.read_dataframe('out.feather')
_df = df

In [ ]:
# 3
# get the closest sentences to this one

sentence = ''
sentence_vec = to_vector(sentence)

dist = 0.08

for i in range(300):
    df = df[(df['p'+str(i)] > sentence_vec[i]-dist) & (df['p'+str(i)] < sentence_vec[i]+dist)]
    
pd.options.display.max_colwidth = 500
print(df.sentence)

# restore old dataframe state
df = _df